In [8]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasisLong5.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
sex = []
race = []
# sz = []
fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
subs = []
tasks = []

for subtask in basis:
    m = re.search('([^-]+)-(.*)', subtask)
#     m = re.search('([^-]+)', subtask)
    sub = m.group(1)
    task = m.group(2)
    subs.append(sub)
    tasks.append(task)
    s = demo['Sex'][sub] == 'M'
    r = demo['Race'][sub] == 'AA'
#     z = demo['DXGROUP_2'][sub] == 'SZP'
#     if demo['DXGROUP_2'][sub] not in ['SZP', 'NC']:
#         continue
#     sz.append(z)
    sex.append(s)
    race.append(r)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    fc.append(p)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
sex = np.array(sex).astype('int')
race = np.array(race).astype('int')
# sz = np.array(sz).astype('int')
fc = np.stack(fc)

print([a.shape for a in [thetas, jitter, sex, race, fc]])
print(np.mean(1-sex))
print(np.mean(1-race))
# print(np.mean(1-sz))

[(3849, 5, 264), (3849, 5, 264), (3849,), (3849,), (3849, 34716)]
0.5263704858404781
0.5188360613146272


In [9]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps
    
ps = tops(thetas, jitter)
aps = np.mean(ps, axis=1)
res = fc - aps

print(ps.shape)
print(aps.shape)
print(res.shape)

(3849, 5, 34716)
(3849, 34716)
(3849, 34716)


In [10]:
for sub,task,i in zip(subs,tasks,range(len(subs))):
    np.save(f'{demodir}/fc/{sub}_task-{task}ab5_fc.npy', aps[i])
    np.save(f'{demodir}/fc/{sub}_task-{task}res5_fc.npy', res[i])
    
print('Complete')

Complete
